# Basic Keras Deep Learning NLP
inspired from 
* https://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras
* https://medium.com/datadriveninvestor/building-neural-network-using-keras-for-regression-ceee5a9eadff


In [1]:
from sys import platform
if platform == "darwin":
    import plaidml.keras
    plaidml.keras.install_backend()
else:
    import keras
del platform
%who

plaidml	 


In [2]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

###

df = pd.read_hdf('reviews.h5')
display(df.head(3))
display(df.describe())
display(df.info())
max_features=100
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

Y = df.stars
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

###

del pd, Tokenizer, pad_sequences, tokenizer, train_test_split, df
%who

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03


,stars,useful,funny,cool
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,3.654000,1.271000,0.452000,0.524000
std,1.469845,2.380594,1.268584,1.384681
min,1.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000,0.000000
75%,5.000000,1.000000,0.000000,1.000000
max,5.000000,23.000000,14.000000,15.000000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 9 columns):
review_id      1000 non-null object
user_id        1000 non-null object
business_id    1000 non-null object
stars          1000 non-null float64
useful         1000 non-null int64
funny          1000 non-null int64
cool           1000 non-null int64
text           1000 non-null object
date           1000 non-null object
dtypes: float64(1), int64(3), object(5)
memory usage: 78.1+ KB


None

(670, 510) (670,)
(330, 510) (330,)
X	 X_test	 X_train	 Y	 Y_test	 Y_train	 max_features	 plaidml	 


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

embed_dim = 12#8
lstm_out = 19#6

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
#model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae','accuracy'])
print(model.summary())

INFO:plaidml:Opening device "opencl_amd_radeon_pro_560x_compute_engine.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 510, 12)           1200      
_________________________________________________________________
lstm_1 (LSTM)                (None, 19)                2432      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 20        
Total params: 3,652
Trainable params: 3,652
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 128
epochs = 1
model.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size, verbose = 1)

Epoch 1/1


INFO:plaidml:Analyzing Ops: 862 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 2193 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 2901 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 3595 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 4315 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 5061 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 5845 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 6648 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 7478 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 8329 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 9229 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 10210 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 11271 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 12383 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 13585 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 14858 of 58862 operations complete
INFO

640/670 [===========================>..] - ETA: 8s - loss: 15.5831 - mean_absolute_error: 3.6674 - acc: 0.0000e+00 

INFO:plaidml:Analyzing Ops: 1020 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 2193 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 2868 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 3486 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 4136 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 4823 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 5530 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 6276 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 7105 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 7953 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 8860 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 9805 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 10808 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 11778 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 12923 of 58862 operations complete
INFO:plaidml:Analyzing Ops: 14138 of 58862 operations complete
INFO

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)

In [ ]:
y_pred= regressor.predict(X_test)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, y_pred)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()